In [124]:
# task: turn text into non-fluent aphasic speech, based on Misra et al (2022) 
# current: basic code, maybe try to speed up transformation

In [121]:
import spacy
from spacy.matcher import Matcher
import numpy as np
nlp = spacy.load("en_core_web_sm")

In [122]:
text1 = "I thought I was going to finish the 3rd season of the Wire tonight."
#  But there was a commentary on episode 11, so I had to re-watch Middle Ground with the commentary. Hopefully I can finish the season next weekend.

vp_pattern = [ [{'POS': 'VERB', 'OP': '?'},
           {'POS': 'ADV', 'OP': '*'},
           {'POS': 'AUX', 'OP': '*'},
           {'POS': 'VERB', 'OP': '+'}] ] 
matcher = Matcher(nlp.vocab)
matcher.add("Verb phrase", vp_pattern)

doc1 = nlp(text1)

In [125]:
def aphasic_speech(text, doc):
    n = 15
    aphasic_utt = ""
    
    if len(text.split()) <= n:  
        # get NPs
        noun_phrases = set()
        for nc in doc.noun_chunks:
            for nop in [nc, doc[nc.root.left_edge.i:nc.root.right_edge.i+1]]:
                noun_phrases.add(nop.text.strip())                   
        print(noun_phrases)      
        
        # get VPs
        verb_phrases = matcher(doc)
        verb_phrases = [doc[start:end] for _, start, 
                        end in verb_phrases]
        print(verb_phrases)
        
        ratio = len(noun_phrases)/len(verb_phrases)
        print(ratio)
        
        X = np.random.uniform(0,1)
        
        if ratio > 2 and X <= 0.8:
            # skip sentence
            return aphasic_utt
        
        else:
            # dont skip sentence
            for tok in doc:
                if tok.dep_ in ["det", "prep", "cop", "aux"]: 
                    # determiners, prepositions, copulas
                    Y = np.random.uniform(0,1)
                    if Y > 0.9:
                        aphasic_utt += tok.text + " "
                        
                elif tok.pos_ in ["ADJ", "ADV"]:      
                    # adjectives, adverbs
                    Z = np.random.uniform(0,1)
                    if Z > 0.5:
                        aphasic_utt += tok.text + " "
                
                elif tok.pos_ == "VERB":
                    # verbs
                    aphasic_utt += tok.lemma_ + " "
                
                else:
                    # all other pos
                    aphasic_utt += tok.text + " "
    
    return aphasic_utt

In [126]:
spacy.explain('aux')

'auxiliary'

In [127]:
print(aphasic_speech(text1, doc1))

{'the Wire', 'the 3rd season of the Wire', 'the 3rd season', 'I'}
[thought, was going, going, finish]
1.0
I think I go finish season Wire tonight . 
